<a href="https://colab.research.google.com/github/HoshizoraProject/StableDiffusionColabTemplate/blob/main/StableDiffusionColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion WebUI on Colab

### 使用流程

1. 到**下載模型**, **Extensions**, **其他設置**進行設置
2. 左上角工具列 > 執行階段 > 變更執行階段類型 > 硬體加速器改為 GPU
3. 左上角工具列 > 執行階段 > 全部執行
4. 到**啟動 WebUI** 點選 public URL

### 參考資料
https://github.com/ddPn08/automatic1111-colab

https://home.gamer.com.tw/artwork.php?sn=5664550


## 1 - 前置步驟

In [ ]:
# @title 1.1 設定系統參數

time_zone = "Asia/Taipei"  # @param {type: "string"}
use_py10 = False # @param {type:"boolean"}

# 設定時區
! rm /etc/localtime
! ln -s /usr/share/zoneinfo/{time_zone} /etc/localtime
! date

# 安裝 python10
if use_py10 == True:
  ! wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
  ! bash ./py310.sh -b -f -p /usr/local
  ! python -m ipykernel install --name "py310" --user

In [ ]:
# @title 1.2 下載 stable-diffusion-webui
%cd /content
repository_url = "https://github.com/AUTOMATIC1111/stable-diffusion-webui"  # @param {type: "string"}
webui_branch = "22bcc7be428c94e9408f589966c2040187245d81"  # @param {type: "string"}
! git clone {repository_url}
! git clone https://github.com/d930065/sd-webui-colab-styles-patch

%cd /content/stable-diffusion-webui
! git checkout {webui_branch}

%cd /content/stable-diffusion-webui/modules
! patch --binary -i /content/sd-webui-colab-styles-patch/styles.patch

In [ ]:
# @title 1.2 建立雲端資料夾
%cd /content
# @markdown 雲端資料夾名稱
data_dir_name = "StableDiffusionColab"  # @param {type:"string"}
extensions_on_drive = False # @param {type:"boolean"}

data_dir = f"/content/drive/MyDrive/{data_dir_name}"

import os


drive_path = "/content/drive"
from google.colab import drive

try:
    drive.mount(drive_path)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

if os.path.exists(f"{data_dir}/script.pre.sh"):
    ! chmod +x {data_dir}/script.pre.sh
    ! {data_dir}/script.pre.sh

models_path = f"{data_dir}/models"
output_path = f"{data_dir}/outputs"
config_path = f"{data_dir}/config"
scripts_path = f"{data_dir}/scripts"
extensions_file_path = f"{data_dir}/extensions.txt"

extensions_path = f"{data_dir}/extensions"
wildcards_path = f"{data_dir}/wildcards"

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(scripts_path, exist_ok=True)
os.makedirs(extensions_path, exist_ok=True)
os.makedirs(wildcards_path, exist_ok=True)

os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)
os.makedirs(f"{models_path}/VAE", exist_ok=True)
os.makedirs(f"{models_path}/embeddings", exist_ok=True)
os.makedirs(f"{models_path}/hypernetworks", exist_ok=True)
os.makedirs(f"{models_path}/Lora", exist_ok=True)
os.makedirs(f"{models_path}/ControlNet", exist_ok=True)

! cp -Rf stable-diffusion-webui/scripts/* {scripts_path}
! rm -Rf stable-diffusion-webui/scripts && ln -s {scripts_path} stable-diffusion-webui/scripts

for dir in os.listdir(models_path):
    if dir == "embeddings":
        ! rm -Rf stable-diffusion-webui/embeddings && ln -s {models_path}/embeddings stable-diffusion-webui/embeddings
    elif dir != "ControlNet":
        ! rm -Rf stable-diffusion-webui/models/{dir} && ln -s {models_path}/{dir} stable-diffusion-webui/models/{dir}

! rm -Rf stable-diffusion-webui/outputs && ln -s {data_dir}/outputs stable-diffusion-webui/outputs

if extensions_on_drive == True:
  ! rm -Rf stable-diffusion-webui/extensions && ln -s {extensions_path} stable-diffusion-webui/extensions

for filename in ["config.json", "ui-config.json", "styles.csv", "extensions.txt"]:
    ! rm -f stable-diffusion-webui/{filename}
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            ! touch {config_path}/{filename}
    ! ln -s {config_path}/{filename} stable-diffusion-webui/{filename}

In [ ]:
# @title 1.3 選擇升級到新版演算系統
%cd /content

use_torch2 = False # @param {type:"boolean"}
update_xformers = False # @param {type:"boolean"}

if use_torch2 == True:
  ! pip install --force-reinstall torch torchvision --index-url https://download.pytorch.org/whl/cu118

if update_xformers == True:
  ! pip install xformers==0.0.17

## 2 - 下載模型

In [ ]:
# @title 2.1 下載SD模型
# @markdown 需要下載SD模型組合包嗎？
need_to_download_sd = False # @param {type:"boolean"}
# @markdown 選擇SD模型組合包
model_package = "AnythingV4.5" # @param ["AnythingV3", "AnythingV4.5", "AbyssOrangeMix3", "CounterfeitV2.5", "PastalMix", "CamelliaMix_2.5D", "ChilloutMix_Ni_fix", "其他(下方填寫)"] 
# @markdown 其他SD模型下載網址
sd_url = "" #@param {type:"string"}
vae_url = "" #@param {type:"string"}
# @markdown ---
# @markdown - 除了利用Colab下載模型，推薦使用雲端硬碟捷徑以節省空間。
# @markdown  - [[AI繪圖] WebUI Colab玩家小技巧：使用雲端硬碟捷徑節省空間](https://home.gamer.com.tw/artwork.php?sn=5677784)
# @markdown - Anything, AbyssOrangeMix, Counterfeit 開啟 VAE 圖片色彩比較豐富
# @markdown  - SD VAE > anime.vae.pt
# @markdown - CamelliaMix_2.5D 開啟 VAE 圖片色彩比較豐富
# @markdown  - SD VAE > vae-ft-mse-840000-ema-pruned.safetensors
# @markdown - PastalMix 和 ChilloutMix 不需要設定 VAE
# @markdown  - SD VAE > None

sd_urls = []
vae_urls = []
if model_package == "AnythingV3":
    sd_urls.append("https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp32-pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/VAE/anime.vae.pt")
elif model_package == "AnythingV4.5":
    sd_urls.append("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/VAE/anime.vae.pt")
elif model_package == "AbyssOrangeMix3":
    sd_urls.append("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/VAE/anime.vae.pt")
elif model_package == "CounterfeitV2.5":
    sd_urls.append("https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_pruned.safetensors")
    vae_urls.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/VAE/anime.vae.pt")
elif model_package == "PastalMix":
    sd_urls.append("https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-better-vae-fp32.safetensors")
elif model_package == "CamelliaMix_2.5D":
    sd_urls.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/SD/CamelliaMix_2.5D/camelliamix25D_v10.safetensors")
    vae_urls.append("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors")
elif model_package == "ChilloutMix_Ni_fix":
    sd_urls.append("https://civitai.com/api/download/models/11745")
elif model_package == "其他(下方填寫)":
    sd_urls.append(sd_url)
    vae_urls.append(vae_url)

if sd_urls != [] and need_to_download_sd == True:
    %cd {models_path}/Stable-diffusion
    for sd_url in sd_urls:
        if sd_url != "":
            ! wget -nc --content-disposition {sd_url}

if vae_urls != [] and need_to_download_sd == True:
    %cd {models_path}/VAE
    for vae_url in vae_urls:
        if vae_url != "":  
            ! wget -nc --content-disposition {vae_url}

In [ ]:
# @title 2.2 下載 Embedding + Hypernetwork + LoRA

# @markdown Embedding
EasyNegative = False # @param {type:"boolean"}
embedding_urls = ""  # @param {type:"string"}

# @markdown Hypernetwork
hypernetwork_urls = ""  # @param {type:"string"}

# @markdown LoRA
koreanDollLikeness_v10 = False # @param {type:"boolean"}
koreanDollLikeness_v15 = False # @param {type:"boolean"}
taiwanDollLikeness_v10 = False # @param {type:"boolean"}
japaneseDollLikeness_v10 = False # @param {type:"boolean"}
lora_urls = ""  # @param {type:"string"}

# @markdown ---
# @markdown - [EasyNegative](https://civitai.com/models/7808/easynegative)
# @markdown   - 泛用負面 embedding
# @markdown   - 觸發詞為 EasyNegative, 加在負面 prompt
# @markdown - DollLikeness 系列
# @markdown   - 觸發詞為 woman, girl
# @markdown   - 模型權重建議為 0.5
# @markdown - 網址之間用英文逗號隔開。例：`url1`, `url2`, `url3`
# @markdown - [LoRA預覽圖](https://i.imgur.com/XooSV8p.png)請自行添加，格式為：`模型名稱.png`

embeddings = []
embedding_urls = list(map(str.strip, embedding_urls.split(",")))
if EasyNegative == True:
    embeddings.append("https://civitai.com/api/download/models/9208")
if embedding_urls != ['']:
    embeddings = embeddings + embedding_urls

if embeddings != []:
    %cd {models_path}/embeddings
    for embedding in embeddings:
        ! wget -nc --content-disposition {embedding}

hypernetworks = []
hypernetwork_urls = list(map(str.strip, hypernetwork_urls.split(",")))
if hypernetwork_urls != ['']:
    hypernetworks = hypernetworks + hypernetwork_urls

if hypernetworks != []:
    %cd {models_path}/hypernetworks
    for hypernetwork in hypernetworks:
        ! wget -nc --content-disposition {hypernetwork}

loras = []
lora_urls = list(map(str.strip, lora_urls.split(",")))
if koreanDollLikeness_v10 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/koreanDollLikeness_v10.safetensors")
if koreanDollLikeness_v15 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/koreanDollLikeness_v15.safetensors")
if taiwanDollLikeness_v10 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/taiwanDollLikeness_v10.safetensors")
if japaneseDollLikeness_v10 == True:
    loras.append("https://huggingface.co/WuLing/SD-WebUI-TW/resolve/main/LoRA/DollLikeness/japaneseDollLikeness_v10.safetensors")
if lora_urls != ['']:
    loras = loras + lora_urls

if loras != []:
    %cd {models_path}/Lora
    for lora in loras:
        ! wget -nc --content-disposition {lora}

## 3 - Extensions

In [ ]:
# @title 3.1 下載 Extensions
locon = True # @param {type:"boolean"}
localization_zh_TW = True # @param {type:"boolean"}
bilingual_localization = False # @param {type:"boolean"}
tag_complete = True # @param {type:"boolean"}
ultimate_upscale = True # @param {type:"boolean"}
image_browser = False # @param {type:"boolean"}
cutoff = True # @param {type:"boolean"}
control_net = False # @param {type:"boolean"}
openpose_editor = False # @param {type:"boolean"}
posex = False # @param {type:"boolean"}
depth_lib = False # @param {type:"boolean"}
latent_couple = False # @param {type:"boolean"}
composable_lora = False # @param {type:"boolean"}
dynamic_prompts = True # @param {type:"boolean"}
lora_block_weight = False # @param {type:"boolean"}
multidiffusion_upscaler = False # @param {type:"boolean"}
webui_artists_ui_tweak = False # @param {type:"boolean"}

extension_urls = ""  # @param {type:"string"}

# @markdown ---
# @markdown - [[AI繪圖] Stable Diffusion WebUI Colab TW Extensions整理](https://home.gamer.com.tw/artwork.php?sn=5678466)
# @markdown - 網址之間用英文逗號隔開。例：`url1`, `url2`, `url3`
# @markdown - 如果使用WebUI內建方法下載 Extension，仍然要執行 `下載 Extensions` 以記錄下載的 Extension。
# @markdown - 如果不想再下載某個 Extension，請刪掉 `config/extensions.txt` 上對應的 GitHub 網址。
# @markdown ---

extensions_pair = [   
            (locon, "https://github.com/KohakuBlueleaf/a1111-sd-webui-locon"),
            (localization_zh_TW, "https://github.com/benlisquare/stable-diffusion-webui-localization-zh_TW"),
            (bilingual_localization, "https://github.com/journey-ad/sd-webui-bilingual-localization"),
            (tag_complete, "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"),
            (ultimate_upscale, "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111"),
            (image_browser, "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser"),
            (cutoff, "https://github.com/hnmr293/sd-webui-cutoff"),
            (control_net, "https://github.com/Mikubill/sd-webui-controlnet"),
            (openpose_editor, "https://github.com/fkunn1326/openpose-editor"),
            (posex, "https://github.com/hnmr293/posex"),
            (depth_lib, "https://github.com/jexom/sd-webui-depth-lib"),
            (latent_couple, "https://github.com/ashen-sensored/stable-diffusion-webui-two-shot"),
            (composable_lora, "https://github.com/opparco/stable-diffusion-webui-composable-lora"),
            (dynamic_prompts, "https://github.com/adieyal/sd-dynamic-prompts"),
            (multidiffusion_upscaler, "https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111"),
            (webui_artists_ui_tweak, "https://github.com/GeorgLegato/stable-diffusion-webui-artists-ui-tweak"),
            (lora_block_weight,"https://github.com/hako-mikan/sd-webui-lora-block-weight")
          ]

with open('/content/stable-diffusion-webui/extensions.txt', 'r') as f:
    extensions_txt = f.read()
extensions_txt = list(map(str.strip, extensions_txt.split("\n")))

extension_urls = list(map(str.strip, extension_urls.split(",")))

extensions = []
for url in extensions_txt:
    if url != '' and url not in extensions:
        extensions.append(url)   
for name, url in extensions_pair:
    if name == True and url not in extensions:
        extensions.append(url)
for url in extension_urls:
    if url != '' and url not in extensions:
        extensions.append(url)

import os

if extensions != []:
    %cd /content/stable-diffusion-webui/extensions
    for extension in extensions:
        extension_name, _ = os.path.splitext(extension.split("/")[-1])
        if not os.path.exists(extension_name):
            ! git clone {extension}
            #if extension_name == "a1111-sd-webui-tagcomplete":
            #    %cd a1111-sd-webui-tagcomplete
            #    ! git checkout 95ebde9fce4a9713436d8b7d4070c18d98d40153
            #    %cd ..

%cd /content/stable-diffusion-webui/extensions
if os.path.exists("sd-webui-controlnet") and not os.path.islink("sd-webui-controlnet/models"):
    ! cp sd-webui-controlnet/models/cldm_v15.yaml {models_path}/ControlNet/cldm_v15.yaml
    ! cp sd-webui-controlnet/models/cldm_v21.yaml {models_path}/ControlNet/cldm_v21.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_depth_sd14v1.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_keypose_sd14v1.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_openpose_sd14v1.yaml
    ! cp sd-webui-controlnet/models/image_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_seg_sd14v1.yaml
    ! cp sd-webui-controlnet/models/sketch_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_canny_sd14v1.yaml
    ! cp sd-webui-controlnet/models/sketch_adapter_v14.yaml {models_path}/ControlNet/t2iadapter_sketch_sd14v1.yaml
    ! cp sd-webui-controlnet/models/t2iadapter_color_sd14v1.yaml {models_path}/ControlNet/t2iadapter_color_sd14v1.yaml
    ! cp sd-webui-controlnet/models/t2iadapter_style_sd14v1.yaml {models_path}/ControlNet/t2iadapter_style_sd14v1.yaml
    ! rm -Rf sd-webui-controlnet/models && ln -s {models_path}/ControlNet sd-webui-controlnet/models

%cd /content/stable-diffusion-webui/extensions
! echo -n '' > /content/stable-diffusion-webui/extensions.txt
for dir in next(os.walk("/content/stable-diffusion-webui/extensions"))[1]:
    %cd {dir}
    if os.path.exists(".git"):
        ! git config --get remote.origin.url >> /content/stable-diffusion-webui/extensions.txt
    %cd ..

# link wildcards
%cd /content
if (os.path.exists("stable-diffusion-webui/extensions/stable-diffusion-webui-wildcards")):
  ! rm -Rf stable-diffusion-webui/extensions/stable-diffusion-webui-wildcards/wildcards
  ! ln -s {wildcards_path} stable-diffusion-webui/extensions/stable-diffusion-webui-wildcards/wildcards
if (os.path.exists("stable-diffusion-webui/extensions/sd-dynamic-prompts")):
  ! rm -Rf stable-diffusion-webui/extensions/sd-dynamic-prompts/wildcards 
  ! ln -s {wildcards_path} stable-diffusion-webui/extensions/sd-dynamic-prompts/wildcards

In [ ]:
# @title 3.2 下載 ControlNet 模型
# @markdown ControlNet
control_canny = False # @param {type:"boolean"}
control_depth = False # @param {type:"boolean"}
control_hed = False # @param {type:"boolean"}
control_mlsd = False # @param {type:"boolean"}
control_normal = False # @param {type:"boolean"}
control_openpose = False # @param {type:"boolean"}
control_scribble = False # @param {type:"boolean"}
control_seg = False # @param {type:"boolean"}
# @markdown T2I Adapter
t2iadapter_canny = False # @param {type:"boolean"}
t2iadapter_color = False # @param {type:"boolean"}
t2iadapter_depth = False # @param {type:"boolean"}
t2iadapter_keypose = False # @param {type:"boolean"}
t2iadapter_openpose = False # @param {type:"boolean"}
t2iadapter_seg = False # @param {type:"boolean"}
t2iadapter_sketch = False # @param {type:"boolean"}
t2iadapter_style = False # @param {type:"boolean"}
# @markdown ---
# @markdown - 除了利用Colab下載模型，推薦使用雲端硬碟捷徑以節省空間。
# @markdown  - [[AI繪圖] WebUI Colab玩家小技巧：使用雲端硬碟捷徑節省空間](https://home.gamer.com.tw/artwork.php?sn=5677784)
# @markdown - 多重 ControlNet 請到 Settings > ControlNet > Multi ControlNet 設置
# @markdown - 參考資料
# @markdown  - https://github.com/Mikubill/sd-webui-controlnet
# @markdown  - https://github.com/lllyasviel/ControlNet
# @markdown  - https://github.com/TencentARC/T2I-Adapter
# @markdown  - [Stable diffusion ControlNet使用心得](https://home.gamer.com.tw/artwork.php?sn=5662905)

import os

control_net_models = []
t2iadapter_models = []

if control_canny == True:
    control_net_models.append("canny")
if control_depth == True:
    control_net_models.append("depth")
if control_hed == True:
    control_net_models.append("hed")
if control_mlsd == True:
    control_net_models.append("mlsd")
if control_normal == True:
    control_net_models.append("normal")
if control_openpose == True:
    control_net_models.append("openpose")
if control_scribble == True:
    control_net_models.append("scribble")
if control_seg == True:
    control_net_models.append("seg")

if t2iadapter_canny == True:
    t2iadapter_models.append("canny")
if t2iadapter_color == True:
    t2iadapter_models.append("color")
if t2iadapter_depth == True:
    t2iadapter_models.append("depth")
if t2iadapter_keypose == True:
    t2iadapter_models.append("keypose")
if t2iadapter_openpose == True:
    t2iadapter_models.append("openpose")
if t2iadapter_seg == True:
    t2iadapter_models.append("seg")
if t2iadapter_sketch == True:
    t2iadapter_models.append("sketch")
if t2iadapter_style == True:
    t2iadapter_models.append("style")

%cd /content/stable-diffusion-webui/extensions

if os.path.exists('sd-webui-controlnet'):
    %cd sd-webui-controlnet/models
    for control_net_model in control_net_models:
        ! wget -nc --content-disposition https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_{control_net_model}-fp16.safetensors
    for t2iadapter_model in t2iadapter_models:
        ! wget -nc --content-disposition https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_{t2iadapter_model}_sd14v1.pth        

In [ ]:
# @title 3.3 下載檔案到指定資料夾
target_dir = "" # @param {type:"string"} 
file_urls = "" # @param {type:"string"}
# @markdown ---
# @markdown - 將指定資料夾填在 `target_dir`，檔案的下載網址填在 `file_urls`。
# @markdown - 網址之間用英文逗號隔開。例：`url1`, `url2`, `url3`
file_urls = list(map(str.strip, file_urls.split(",")))

import os
if os.path.exists(os.path.split(target_dir)[0]):
    os.makedirs(target_dir, exist_ok=True)
    %cd {target_dir}
    for url in file_urls:
        if url != '':
            ! wget -nc --content-disposition {url}

## 4 - 其他設置

In [ ]:
# @title 4.1 config.json
recommended_settings = True # @param {type:"boolean"}

import json

def read_config(filename):
  with open(filename, "r") as f:
    config = json.load(f)
  return config

def write_config(filename, config):
  with open(filename, "w") as f:
    json.dump(config, f, indent=4)

if recommended_settings == True:
    %cd /content/stable-diffusion-webui/extensions
    config_file = f"{config_path}/config.json"
    config = read_config(config_file)
    if "CLIP_stop_at_last_layers" not in config or config["CLIP_stop_at_last_layers"] == 1:
        config["CLIP_stop_at_last_layers"] = 2
    if "quicksettings" not in config or config["quicksettings"] == "sd_model_checkpoint":
        config["quicksettings"] = "sd_model_checkpoint, sd_vae"    
    if os.path.exists("sd-webui-controlnet") and ("control_net_max_models_num" not in config or config["control_net_max_models_num"] == 1): 
        config["control_net_max_models_num"] = 3
    if os.path.exists("stable-diffusion-webui-localization-zh_TW"):
        if not os.path.exists("sd-webui-bilingual-localization") and ("localization" not in config or config["localization"] == "None"): 
            config["localization"] = "zh_TW"
        elif os.path.exists("sd-webui-bilingual-localization") and ("bilingual_localization_file" not in config or config["bilingual_localization_file"] == "None"):
            config["localization"] = "None"
            config["bilingual_localization_file"] = "zh_TW"
    write_config(config_file, config)  

In [ ]:
# @title 4.2 命令行參數
# @markdown ---
# @markdown 如果 public URL 有問題可以嘗試改用 ngrok
# @markdown - 註冊 [ngrok](https://ngrok.com/) 帳號
# @markdown - Your Authtoken > Copy
# @markdown - 勾選 ngrok > 貼上 authtoken
# @markdown - 等 local URL 出來之後再點 ngrok URL
ngrok = False # @param {type:"boolean"}
your_authtoken = "" # @param {type:"string"}

# @markdown ---
# @markdown 底下是一可以選擇開關的選項, 如果不熟悉請維持原本設定<br>
# @markdown 設定參考 [Command Line Arguments](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings)<br>
# @markdown 設定參考 [Optimizations](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Optimizations)
# @markdown * 不使用半精度運算VAE, 
no_half_vae = True # @param {type:"boolean"} 
# @markdown * 不使用半精度運算模型, 會增加VRAM消耗, 但會比較細緻
no_half = True # @param {type:"boolean"}
# @markdown * 一種加速推算的模式, 使用 Tensor Cores
opt_channelslast = True # @param {type:"boolean"}
# @markdown ---
# @markdown 這裡面的選項只能選只勾選一個就好
# @markdown * 啟用 xformers 減少VRAM消耗, 但會犧牲一些細節. 泛用性很高.
xformers = True # @param {type:"boolean"}
# @markdown * 比 xformers 更快更好(理論上). 請在 1.3 把 `use_torch2` 勾選才有用, 不然會豹炸
opt_sdp_attention = False # @param {type:"boolean"}
# @markdown ---
# @markdown ###### 其他參數：請確認清楚你在做什麼, 不然請放空
outher_args = "" # @param {type:"string"}

import os

os.environ["COMMANDLINE_ARGS"] = "--gradio-queue --no-hashing --enable-insecure-extension-access --disable-safe-unpickle --theme dark"

if no_half_vae:
  os.environ["COMMANDLINE_ARGS"] += " --no-half-vae"

if no_half:
  os.environ["COMMANDLINE_ARGS"] += " --no-half"

if xformers:
  os.environ["COMMANDLINE_ARGS"] += " --xformers"

if opt_sdp_attention:
  os.environ["COMMANDLINE_ARGS"] += " --opt-sdp-attention"

if opt_channelslast:
  os.environ["COMMANDLINE_ARGS"] += " --opt-channelslast"

outher_args = outher_args.strip()
if len(outher_args) > 0:
  os.environ["COMMANDLINE_ARGS"] += (" " + outher_args)

if ngrok == True:
  os.environ["COMMANDLINE_ARGS"] += (" --ngrok " + your_authtoken)
else:
  os.environ["COMMANDLINE_ARGS"] += " --share"

## 5 - 啟動 WebUI

In [ ]:
# @title 5.1 啟動 WebUI
# @markdown - 點選 public URL 開啟 WebUI<br>
# @markdown ![](https://i.imgur.com/y3xGiIX.png)

%cd /content/stable-diffusion-webui/
! python launch.py